In [10]:
import os
import pandas as pd
import numpy as np
from scipy import interpolate


# 定义文件夹路径


In [11]:
ds_root_folder= 'D:\data_guan_cut_sub\DS'  # DS 文件夹路径
smarteye_root_folder = 'D:\data_guan_cut_sub\SmartEye'  # SmartEye 主文件夹路径（包含 SmartEye-No1 ~ SmartEye-No39）
output_folder = 'D:\data_guan_cut_sub\interpolated'  # 插值后文件的输出路径


<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
C:\Users\goodb\AppData\Local\Temp\ipykernel_19688\2671667195.py:1: SyntaxWarning: invalid escape sequence '\d'
  ds_root_folder= 'D:\data_guan_cut_sub\DS'  # DS 文件夹路径
C:\Users\goodb\AppData\Local\Temp\ipykernel_19688\2671667195.py:2: SyntaxWarning: invalid escape sequence '\d'
  smarteye_root_folder = 'D:\data_guan_cut_sub\SmartEye'  # SmartEye 主文件夹路径（包含 SmartEye-No1 ~ SmartEye-No39）
C:\Users\goodb\AppData\Local\Temp\ipykernel_19688\2671667195.py:3: SyntaxWarning: invalid escape sequence '\d'
  output_folder = 'D:\data_guan_cut_sub\interpolated'  # 插值后文件的输出路径


In [15]:
# 获取 DS 和 SmartEye 主文件夹中的实验者文件夹列表
# ds_folders = [f for f in os.listdir(ds_root_folder) if os.path.isdir(os.path.join(ds_root_folder, f))]
# smarteye_folders = [f for f in os.listdir(smarteye_root_folder) if os.path.isdir(os.path.join(smarteye_root_folder, f))]
ds_folders = ['No38', 'No39', 'No4', 'No40', 'No41', 'No42', 'No43']
smarteye_folders = ['No38', 'No39', 'No4', 'No40', 'No41', 'No42', 'No43']
print(ds_folders)
print(smarteye_folders)

['No38', 'No39', 'No4', 'No40', 'No41', 'No42', 'No43']
['No38', 'No39', 'No4', 'No40', 'No41', 'No42', 'No43']


# 遍历每个实验者的文件夹


In [17]:
for ds_folder in ds_folders:
    participant_id = ds_folder.replace('No', '')  # 提取文件夹中的数字部分，例如从 'No1' 提取 '1'
    smarteye_folder = f"No{participant_id}"  # SmartEye 对应的文件夹名

    # 检查对应的 SmartEye 文件夹是否存在
    if smarteye_folder not in smarteye_folders:
        print(f"未找到与 {ds_folder} 对应的 SmartEye 文件夹")
        continue

    ds_folder_path = os.path.join(ds_root_folder, ds_folder)
    smarteye_folder_path = os.path.join(smarteye_root_folder, smarteye_folder)

    # 获取每个实验者文件夹中的 Excel 文件列表
    ds_file = [f for f in os.listdir(ds_folder_path) if f.endswith('.xlsx')][0]
    smarteye_files = [f for f in os.listdir(smarteye_folder_path) if f.endswith('.xlsx')]

    linear = {'EyelidOpening','GazeDirectionX','GazeDirectionY','PupilDiameter','movingGazeRatio'}
    nearest = {'Blink','Fixation','GazeObjects','Saccade','ScreenName'}

    # 遍历 DS 文件
    for smarteye_file in smarteye_files:
        smarteye_file_list = smarteye_file.split('_')
        datatype = smarteye_file_list[1]
        ds_file_path = os.path.join(ds_folder_path, ds_file)
        smarteye_file_path = os.path.join(smarteye_folder_path, smarteye_file)

        # 读取 DS 和 SmartEye 文件中的所有 sheet
        ds_data = pd.read_excel(ds_file_path, sheet_name=None)
        smarteye_data = pd.read_excel(smarteye_file_path, sheet_name=None)

        # 创建一个字典来保存插值后的 SmartEye 数据
        interpolated_smarteye_data = {}

        # 遍历 DS 的每个 sheet，按 DS 的行数对 SmartEye 进行插值
        for sheet_name, ds_df in ds_data.items():
            if sheet_name not in smarteye_data:
                print(f"SmartEye 文件中未找到对应的 sheet: {sheet_name}")
                continue

            # 获取 DS 文件的目标行数
            ds_row_count = ds_df.shape[0]

            # 获取 SmartEye 文件的行数
            smarteye_df = smarteye_data[sheet_name]
            smarteye_row_count = smarteye_df.shape[0]

            # 创建一个目标索引数组，行数为 DS 文件的行数
            smarteye_index = np.arange(smarteye_row_count)
            target_index = np.linspace(0, smarteye_row_count - 1, ds_row_count)

            # 创建一个空的 DataFrame 来保存插值后的结果
            interpolated_df = pd.DataFrame()

            # 对 SmartEye 数据的每一列进行插值
            for col in smarteye_df.columns:
                y = smarteye_df[col].values

                # 创建插值函数，线性插值
                kind = 'linear' if datatype in linear else 'nearest'
                f = interpolate.interp1d(smarteye_index, y, kind=kind, fill_value="extrapolate")

                # 计算插值后的结果
                interpolated_values = f(target_index)

                # 将插值结果保存到新的 DataFrame 中
                interpolated_df[col] = interpolated_values

            # 保存插值后的数据
            interpolated_smarteye_data[sheet_name] = interpolated_df

        # 将插值后的 SmartEye 数据保存到新的 Excel 文件中
        output_file_name = f"{smarteye_file[:-5]}_{kind}.xlsx"
        output_file_path = os.path.join(output_folder, smarteye_folder, output_file_name)
        
        with pd.ExcelWriter(output_file_path) as writer:
            for sheet_name, df in interpolated_smarteye_data.items():
                df.to_excel(writer, sheet_name=sheet_name, index=False)

        print(f"已处理并保存文件: {output_file_name}")

已处理并保存文件: No38_Blink_eachLaneChange_nearest.xlsx
已处理并保存文件: No38_EyelidOpening_Change_eachLaneChange_linear.xlsx
已处理并保存文件: No38_EyelidOpening_eachLaneChange_linear.xlsx
已处理并保存文件: No38_Fixation_recalculated(20240104)_eachLaneChange_nearest.xlsx
已处理并保存文件: No38_GazeDirectionX_eachLaneChange_linear.xlsx
已处理并保存文件: No38_GazeDirectionY_eachLaneChange_linear.xlsx
已处理并保存文件: No38_GazeObjects_recalculated(Reidentified+gap_filled)_eachLaneChange_nearest.xlsx
已处理并保存文件: No38_movingGazeRatio_eachLaneChange_transposed_linear.xlsx
已处理并保存文件: No38_PupilDiameter_Change_eachLaneChange_linear.xlsx
已处理并保存文件: No38_PupilDiameter_eachLaneChange_linear.xlsx
已处理并保存文件: No38_Saccade_recalculated(20231231)_eachLaneChange_nearest.xlsx
已处理并保存文件: No38_ScreenName_eachLaneChange_nearest.xlsx
已处理并保存文件: No39_Blink_eachLaneChange_nearest.xlsx
已处理并保存文件: No39_EyelidOpening_Change_eachLaneChange_linear.xlsx
已处理并保存文件: No39_EyelidOpening_eachLaneChange_linear.xlsx
已处理并保存文件: No39_Fixation_recalculated(20240104)_eachLaneChange_near